In [ ]:
import cv2
from ultralytics import YOLO
import pandas as pd
import time
from datetime import datetime
import numpy as np


path_output_video = "res//output_video.mp4"
path_model = "yolov9c.pt"

model = YOLO(path_model)


cap = cv2.VideoCapture(0)
assert cap.isOpened(), "Error opening webcam"


frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))


video_writer = cv2.VideoWriter(path_output_video, cv2.VideoWriter_fourcc(*"mp4v"), 30, (frame_width, frame_height))


count_line_y = frame_height // 2


count = 0
tracked_objects = {}

start_time = time.time()
current_minute = -1
df = pd.DataFrame(columns=['Time', 'Count'])


cv2.namedWindow('Processed Feed', cv2.WINDOW_NORMAL)

def create_count_display(df):
    """Create a visual display of the count data"""
    height = 400
    width = 300
    display = np.ones((height, width, 3), dtype=np.uint8) * 255
    

    cv2.putText(display, "Count by Minute", (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)

    cv2.putText(display, "Time", (10, 70), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)
    cv2.putText(display, "Count", (200, 70), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)

    cv2.line(display, (10, 80), (width-10, 80), (0, 0, 0), 1)
    

    last_entries = df.tail(10)
    for i, (_, row) in enumerate(last_entries.iterrows()):
        y_pos = 110 + i * 30
        time_str = row['Time'].strftime('%H:%M')
        cv2.putText(display, time_str, (10, y_pos), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)
        cv2.putText(display, str(row['Count']), (200, y_pos), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)
    
    return display

# Main loop
while True:
    success, frame = cap.read()
    if not success:
        print("Failed to grab frame from webcam")
        break


    elapsed_time = time.time() - start_time
    new_minute = int(elapsed_time // 60)

    
    if new_minute > current_minute:
        new_row = pd.DataFrame({
            'Time': [datetime.now().replace(second=0, microsecond=0)],
            'Count': [count]
        })
        df = pd.concat([df, new_row], ignore_index=True)
        current_minute = new_minute
        df.to_csv('count_data_realtime.csv', index=False)


    results = model.track(frame, persist=True, tracker='botsort.yaml', iou=0.2)

   
    if results[0].boxes.id is not None:
        boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
        track_ids = results[0].boxes.id.cpu().numpy().astype(int)
        class_ids = results[0].boxes.cls.cpu().numpy().astype(int)
        
        for box, track_id, class_id in zip(boxes, track_ids, class_ids):
            if class_id == 0:  
                x1, y1, x2, y2 = box
                center_y = (y1 + y2) // 2
                
                if track_id in tracked_objects:
                    prev_center_y = tracked_objects[track_id]
                    if prev_center_y < count_line_y and center_y >= count_line_y:
                        count += 1
                    elif prev_center_y > count_line_y and center_y <= count_line_y:
                        count = max(0, count - 1)
                
                tracked_objects[track_id] = center_y
                
           
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f"ID: {track_id}", (x1, y1 - 10), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

  
    cv2.line(frame, (0, count_line_y), (frame_width, count_line_y), (255, 0, 0), 2)

    # Display count and time
    cv2.putText(frame, f"Count: {count}", (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.putText(frame, f"Time: {int(elapsed_time)}s", (10, 70), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)


    count_display = create_count_display(df)
    cv2.imshow('Count Data', count_display)

    video_writer.write(frame)
    cv2.imshow('Processed Feed', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

final_row = pd.DataFrame({
    'Time': [datetime.now().replace(second=0, microsecond=0)],
    'Count': [count]
})
df = pd.concat([df, final_row], ignore_index=True)


cap.release()
video_writer.release()
cv2.destroyAllWindows()


print("\nFinal count data:")
print(df)
print(f'\nSaved output video to {path_output_video}')
print(f'Saved count data to count_data_realtime.csv')

C:\Users\shehan.hirusha\AppData\Local\Temp\ipykernel_14356\2011653265.py:85: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_row], ignore_index=True)
